# Network Intrusion Detection with Deep Learning

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


## Open Data

In [0]:
import numpy as np
def load_matrices(file_name):
    with open(file_name, 'rb') as f:
        A = np.load(f)
        B = np.load(f)
        C = np.load(f)
        D = np.load(f)
        E = np.load(f)
        F = np.load(f)
    return (A,B,C,D,E,F)

In [0]:
my_file = '../data_NSL/data.npy'
train, test, test2, y_train, y_test, y_test2 = load_matrices(my_file)

## The Model

In [10]:
# talos item
import talos as ta
import wrangle as wr
from talos.metrics.keras_metrics import fmeasure_acc
from talos import live




# Keras items
from keras import backend as K
from keras.layers import Input, Dense 
from keras.layers import InputLayer
from keras.models import Model, Sequential
from keras.layers import Activation, Reshape, Dropout
from keras.optimizers import Adam
from keras.layers.embeddings import Embedding
from keras.losses import binary_crossentropy
from keras.callbacks import EarlyStopping
from keras.activations import relu, elu
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import PReLU
from keras.callbacks import TensorBoard
from keras.models import load_model



%matplotlib inline

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [0]:
def selu(x):
    """Scaled Exponential Linear Unit. (Klambauer et al., 2017)
    # Arguments
        x: A tensor or variable to compute the activation function for.
    # References
        - [Self-Normalizing Neural Networks](https://arxiv.org/abs/1706.02515)
    """
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale * elu(x, alpha)

#model.add(Dense(32, input_shape=(input_length - 1,)), activation=selu)

In [13]:
# then we load the dataset
x=train
y=y_train
# and normalize every feature to mean 0, std 1
#x = wr.mean_zero(pd.DataFrame(x)).values
input_dim = train.shape[1]
print(input_dim)

122


In [0]:
from keras.callbacks import ModelCheckpoint
filepath="./log5pr/taloslr_ae_weights_lr_pr.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:

filepath2="./log5pr/taloslr_dnn_weights_lr_pr.best.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='val_acc', save_best_only=True, mode='max')
callbacks_list2 = [checkpoint2]

In [0]:
#actvation leakyrelu
LR= LeakyReLU(0.01)
LR.__name__ = 'lrelu'
#actvation prelu
#PR= PReLU()
#PR.__name__ = 'prelu'

In [0]:
# first we have to make sure to input data and params into the function
# add input parameters to the function
def dnn(x_train, y_train, x_val, y_val, params):

    input_dim = x_train.shape[1]

    input_img = Input(shape=(input_dim,))

    first_neuron=params['first_neuron']
    second_neuron=params['second_neuron']
    third_neuron=params['third_neuron']
    batch_size=params['batch_size']
    kernel_initial=params['kernel_initializer']
    lr=params['lr']
    activation=params['activation']
    
    print('activation=',activation)

    ###AE
    
    # replace the hyperparameter inputs with references to params dictionary 
    autoencoder = Sequential()
    #encoder
    
    if activation==PReLU: 
        autoencoder.add(Dense(first_neuron, input_shape=(input_dim,), kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        autoencoder.add(Dense(second_neuron, kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        autoencoder.add(Dense(third_neuron, kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
                   
        #decoder
        #model.add(Dense(8, activation=params['activation']))
        autoencoder.add(Dense(second_neuron, kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        autoencoder.add(Dense(first_neuron, kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        autoencoder.add(Dense(input_dim, activation='sigmoid', kernel_initializer=kernel_initial))
        
    else:
        autoencoder.add(Dense(first_neuron, input_shape=(input_dim,), activation=activation, kernel_initializer=kernel_initial))
        autoencoder.add(Dense(second_neuron, activation=activation, kernel_initializer=kernel_initial))
        autoencoder.add(Dense(third_neuron, activation=activation, kernel_initializer=kernel_initial))
        #decoder
        #model.add(Dense(8, activation=params['activation']))
        autoencoder.add(Dense(second_neuron, activation=activation, kernel_initializer=kernel_initial))
        autoencoder.add(Dense(first_neuron, activation=activation, kernel_initializer=kernel_initial))
        autoencoder.add(Dense(input_dim, activation='sigmoid', kernel_initializer=kernel_initial))
    
     #model = multi_gpu_model(model, gpus=1)

    
    #model = multi_gpu_model(model, gpus=1)
    autoencoder.summary()
    print(input_dim)

    input_img = Input(shape=(input_dim,))
    encoder_layer1 = autoencoder.layers[0]
    encoder_layer2 = autoencoder.layers[1]
    encoder_layer3 = autoencoder.layers[2]
    encoder = Model(input_img, encoder_layer3(encoder_layer2(encoder_layer1(input_img))))
    
    #encoder_layer1 = autoencoder.layers[0]
    #encoder_layer2 = autoencoder.layers[1]
    #encoder_layer3 = autoencoder.layers[2]
    #encoder = Model(input_img, encoder_layer3(encoder_layer2(encoder_layer1(input_img))))
    encoder.summary()

    ## using binary cross entropy because from 122 features only 32 features (sudah dinormalisasi dalam rentang 0 dan 1) float and 90 features binary
    autoencoder.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    #model.compile(loss=params['losses'],
    #              optimizer=params['optimizer'](),
    #              metrics=['acc', fmeasure_acc])
    
   

    
    history=autoencoder.fit(x,x,
                             epochs=params['epochs'],
                             batch_size=batch_size,
                              #shuffle=True,
                              #validation_split=0.2,
                              callbacks=callbacks_list,                    
                             #callbacks=[early_stopping])
                              validation_data=(test,test),
                              verbose=0)

    num_classes=5
    out2 = Dense(num_classes, activation='softmax',kernel_initializer=kernel_initial)(encoder.output)
    newmodel = Model(encoder.input,out2)
    
   
    newmodel.compile(loss='categorical_crossentropy', 
                     #optimizer='adam', 
                     optimizer= Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08,decay=0.0, amsgrad=False),
                     metrics=['acc']) 
    newmodel.summary()


    history2a=newmodel.fit(x, y,
                           epochs=params['epochs2'],
                           batch_size=batch_size,
                           #validation_split=0.2,
                           callbacks=callbacks_list2,
                           #callbacks=[early_stopping],
                           validation_data=(test,y_test),
                           verbose=0)

    
    del encoder
    del autoencoder
    #del dnn_model
    
    tf.reset_default_graph()
    
    return history2a, newmodel #, encoder
    


In [0]:
# then we can go ahead and set the parameter space
p = {'first_neuron':[100, 80, 75],
     'second_neuron':[60, 50,40],
     'third_neuron':[30,20,15],
     'batch_size': [32,64,256],
     'epochs': [30],
     'epochs2': [50],
     'dropout': [0],
     'kernel_initializer': ['lecun_uniform','lecun_normal','he_normal','he_uniform','normal','glorot_uniform','glorot_normal'],
     'optimizer': [Adam],
     'lr':[0.1,0.01, 0.001,0.0001,0.00001],
     #'activation':[PReLU]}
     'activation':[relu, elu,selu, LR,PReLU]}

In [0]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, clear_session

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

    

In [21]:

t = ta.Scan(x, y, p, dnn, experiment_no='52',grid_downsample=0.05,)

#experiment no 6 3780 kombinasi


  0%|          | 0/113 [00:00<?, ?it/s]

activation= <class 'keras.layers.advanced_activations.PReLU'>
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 80)                9840      
_________________________________________________________________
p_re_lu_6 (PReLU)            (None, 80)                80        
_________________________________________________________________
dense_8 (Dense)              (None, 50)                4050      
_________________________________________________________________
p_re_lu_7 (PReLU)            (None, 50)                50        
_________________________________________________________________
dense_9 (Dense)              (None, 15)                765       
_________________________________________________________________
p_re_lu_8 (PReLU)            (None, 15)                15        
__________________________________________________________


  1%|          | 1/113 [03:28<6:28:45, 208.26s/it]

activation= <class 'keras.layers.advanced_activations.PReLU'>
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 80)                9840      
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 80)                80        
_________________________________________________________________
dense_2 (Dense)              (None, 60)                4860      
_________________________________________________________________
p_re_lu_2 (PReLU)            (None, 60)                60        
_________________________________________________________________
dense_3 (Dense)              (None, 20)                1220      
_________________________________________________________________
p_re_lu_3 (PReLU)            (None, 20)                20        
__________________________________________________________


  2%|▏         | 2/113 [07:01<6:27:55, 209.69s/it]

...

activation= <class 'keras.layers.advanced_activations.PReLU'>
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 80)                9840      
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 80)                80        
_________________________________________________________________
dense_2 (Dense)              (None, 40)                3240      
_________________________________________________________________
p_re_lu_2 (PReLU)            (None, 40)                40        
_________________________________________________________________
dense_3 (Dense)              (None, 30)                1230      
_________________________________________________________________
p_re_lu_3 (PReLU)            (None, 30)                30        
__________________________________________________________


 99%|█████████▉| 112/113 [11:19:16<07:32, 452.80s/it]

activation= <class 'keras.layers.advanced_activations.PReLU'>
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 75)                9225      
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 75)                75        
_________________________________________________________________
dense_2 (Dense)              (None, 60)                4560      
_________________________________________________________________
p_re_lu_2 (PReLU)            (None, 60)                60        
_________________________________________________________________
dense_3 (Dense)              (None, 30)                1830      
_________________________________________________________________
p_re_lu_3 (PReLU)            (None, 30)                30        
__________________________________________________________


100%|██████████| 113/113 [11:29:10<00:00, 495.16s/it]


Scan Finished!


In [22]:
x.shape



(125973, 122)

In [23]:
t.data

,round_epochs,val_loss,val_acc,loss,acc,first_neuron,second_neuron,third_neuron,batch_size,epochs,epochs2,dropout,kernel_initializer,optimizer,lr,activation
1,50,1.5316054570158801,0.8091731724627396,0.005853491276579219,0.9985552459654052,80,50,15,256,30,50,0,lecun_uniform,<class 'keras.optimizers.Adam'>,0.01,<class 'keras.layers.advanced_activations.PReLU'>
2,50,0.984927713913305,0.7948012775017743,0.010141822072645844,0.9973327617822867,80,60,20,256,30,50,0,glorot_uniform,<class 'keras.optimizers.Adam'>,0.0001,<class 'keras.layers.advanced_activations.PReLU'>
3,50,10.785911392201085,0.33081973030518097,10.228756574688711,0.3651020456771642,100,40,20,64,30,50,0,lecun_uniform,<class 'keras.optimizers.Adam'>,0.1,<class 'keras.layers.advanced_activations.PReLU'>
4,50,1.7813539926340767,0.8219925479063165,0.004749547677217855,0.9986505044731807,80,50,20,256,30,50,0,lecun_normal,<class 'keras.optimizers.Adam'>,0.01,<class 'keras.layers.advanced_activations.PReLU'>
5,50,1.5749640406827878,0.8133871540099361,0.006206310389636871,0.9985393695474427,100,40,30,256,30,50,0,normal,<class 'keras.optimizers.Adam'>,0.01,<class 'keras.layers.advanced_activations.PReLU'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50,1.6211636854114897,0.7853530872959545,0.0035215598654201575,0.9988251450707691,80,40,30,64,30,50,0,normal,<class 'keras.optimizers.Adam'>,0.001,<class 'keras.layers.advanced_activations.PReLU'>
110,50,1.9020458109163916,0.8020315826827538,0.02214901978310364,0.9956736761051972,80,50,20,32,30,50,0,he_uniform,<class 'keras.optimizers.Adam'>,0.01,<class 'keras.layers.advanced_activations.PReLU'>
111,50,1.9609854186232671,0.8165809084457062,0.015502197227289632,0.9971739976026609,80,60,30,64,30,50,0,he_normal,<class 'keras.optimizers.Adam'>,0.01,<class 'keras.layers.advanced_activations.PReLU'>
112,50,9.170098151104767,0.4310681334279631,7.370108262700316,0.5424495725302853,80,40,30,32,30,50,0,glorot_normal,<class 'keras.optimizers.Adam'>,0.1,<class 'keras.layers.advanced_activations.PReLU'>


In [0]:
r = ta.Reporting(t)

In [25]:
r.high('val_acc')

'0.8305092264017033'

In [0]:
r.best_params()